<a href="https://colab.research.google.com/github/gabrielycarrari/tcc/blob/pre-processing/pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook para Pré-processamento

# Importações

In [1]:
!pip install datasets
from datasets import load_dataset

In [2]:
!pip install tensorflow==2.11.0
!pip install tensorflow_io==0.31.0
import tensorflow_io as tfio

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [4]:
from huggingface_hub import login
from google.colab import userdata


In [5]:
login(token=userdata.get('HUGGINGFACE_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [6]:
# Carregar o dataset Common Voice 17
dataset = load_dataset("mozilla-foundation/common_voice_17_0", "pt", split='train', trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


### Testes

In [ ]:
dataset[0]

{'client_id': 'df268ebd4060c8106179019a8bb331f8db173023c64d4e56a38a54f77f2fe6706480320965871d1b8d11da18601c241c8c4b80dc44d3ff9a32ee54991f5c0e99',
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/fe3a346df3c979ff3eb48fa107c50894678ab25af4c6f7785dec890ecdff72f7/pt_train_0/common_voice_pt_33954672.mp3',
 'audio': {'path': '/root/.cache/huggingface/datasets/downloads/extracted/fe3a346df3c979ff3eb48fa107c50894678ab25af4c6f7785dec890ecdff72f7/pt_train_0/common_voice_pt_33954672.mp3',
  'array': array([ 1.77635684e-15, -5.41788836e-14, -9.05941988e-14, ...,
          1.72863140e-10,  1.45055787e-10,  7.10012743e-11]),
  'sampling_rate': 48000},
 'sentence': 'Sinta-se feliz com a vitória que você ganha.',
 'up_votes': 4,
 'down_votes': 0,
 'age': 'thirties',
 'gender': 'male_masculine',
 'accent': 'Minas Gerais',
 'locale': 'pt',
 'segment': '',
 'variant': 'Portuguese (Brasil)'}

In [ ]:
dataset[0]['sentence']

'Sinta-se feliz com a vitória que você ganha.'

In [107]:
file = tf.io.read_file('/content/samples_pt_sample.wav')

NotFoundError: {{function_node __wrapped__ReadFile_device_/job:localhost/replica:0/task:0/device:CPU:0}} /content/samples_pt_sample.wav; No such file or directory [Op:ReadFile]

In [ ]:
import librosa
import tensorflow as tf
import soundfile as sf

# Load the audio file with librosa
audio_data, sample_rate = librosa.load('/content/samples_pt_sample.wav', sr=None)

# Resample to 16kHz (if necessary)
target_sample_rate = 16000
if sample_rate != target_sample_rate:
    audio_data = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=target_sample_rate)

# Save the resampled audio as a WAV file
sf.write('resampled_audio.wav', audio_data, target_sample_rate)

# Load the resampled file using tensorflow
file = tf.io.read_file('resampled_audio.wav')
audio, _ = tf.audio.decode_wav(file)

In [ ]:
audio = tf.squeeze(audio, axis=-1)
# 3. Change type to float
audio = tf.cast(audio, tf.float32)

In [ ]:
audio

<tf.Tensor: shape=(74240,), dtype=float32, numpy=
array([2.2583008e-03, 2.4719238e-03, 1.5869141e-03, ..., 3.0517578e-05,
       9.1552734e-05, 9.1552734e-05], dtype=float32)>

In [ ]:
audio2 = dataset[0]['audio']['array']
sampling_rate = dataset[0]['audio']['sampling_rate']

# 2. Convert audio to a tensor
audio2 = tf.convert_to_tensor(audio2, dtype=tf.float32)

In [ ]:
dataset[0]['audio']['sampling_rate']

48000

In [ ]:
audio2

<tf.Tensor: shape=(195264,), dtype=float32, numpy=
array([ 1.7763568e-15, -5.4178884e-14, -9.0594199e-14, ...,
        1.7286314e-10,  1.4505579e-10,  7.1001274e-11], dtype=float32)>

### Definindo Caracteres

In [7]:
# The set of characters accepted in the transcription.
characters = [x for x in "abcdefghijklmnopqrstuvwxyzáàâãéêíóôõúç'?!.- "]

# Mapping characters to integers
char_to_num = keras.layers.StringLookup(vocabulary=characters, oov_token="")

# Mapping integers back to original characters
num_to_char = keras.layers.StringLookup(
    vocabulary=char_to_num.get_vocabulary(), oov_token="", invert=True
)

print(
    f"The vocabulary is: {char_to_num.get_vocabulary()} "
    f"(size={char_to_num.vocabulary_size()})"
)

The vocabulary is: ['', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'á', 'à', 'â', 'ã', 'é', 'ê', 'í', 'ó', 'ô', 'õ', 'ú', 'ç', "'", '?', '!', '.', '-', ' '] (size=45)


# Data Preprocessing

In [8]:
# The window length in samples.
frame_length = 256
# The number of samples to step.
frame_step = 160
# The size of the FFT to apply.
fft_length = 384

In [108]:
file_ = tf.io.read_file(dataset[0]['path'])

In [111]:
audio = tfio.audio.decode_mp3(file_)

NotImplementedError: unable to open file: libtensorflow_io.so, from paths: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/usr/local/lib/python3.10/dist-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZNK10tensorflow4data11DatasetBase8FinalizeEPNS_15OpKernelContextESt8functionIFNS_8StatusOrISt10unique_ptrIS1_NS_4core15RefCountDeleterEEEEvEE']

In [100]:
dataset[0]['path']

'/root/.cache/huggingface/datasets/downloads/extracted/fe3a346df3c979ff3eb48fa107c50894678ab25af4c6f7785dec890ecdff72f7/pt_train_0/common_voice_pt_33954672.mp3'

In [78]:
def encode_single_sample(audio, label):
  ###########################################
  ##  Process the Audio
  ##########################################
  # 1. Read wav file
  file = tf.io.read_file(wavs_path + wav_file + ".wav")
  file = tf.io.read_file()
  # 2. Decode the wav file
  audio, _ = tf.audio.decode_wav(file)
  audio = tf.squeeze(audio, axis=-1)
  # 3. Change type to float
  audio = tf.cast(audio, tf.float32)

  # 2. Convert audio to a tensor
  audio = tf.convert_to_tensor(audio, dtype=tf.float32)

  # 4. Get the spectrogram
  spectrogram = tf.signal.stft(
      audio, frame_length=frame_length, frame_step=frame_step, fft_length=fft_length
  )
  # 5. We only need the magnitude, which can be derived by applying tf.abs
  spectrogram = tf.abs(spectrogram)
  spectrogram = tf.math.pow(spectrogram, 0.5)
  # 6. normalisation
  means = tf.math.reduce_mean(spectrogram, 1, keepdims=True)
  stddevs = tf.math.reduce_std(spectrogram, 1, keepdims=True)
  spectrogram = (spectrogram - means) / (stddevs + 1e-10)


  ###########################################
  ##  Process the label
  ##########################################
  # 7. Convert label to Lower case
  label = tf.strings.lower(label)
  # 8. Split the label
  label = tf.strings.unicode_split(label, input_encoding="UTF-8")
  # 9. Map the characters in label to numbers
  label = char_to_num(label)
  # 10. Return a dict as our model is expecting two inputs
  return spectrogram, label

In [79]:
spectrograms = []
labels = []

for data in tqdm(dataset, desc="Processando dados", unit="data"):
    audio = data['audio']['array']
    spectrogram, label = encode_single_sample(audio, data['sentence'])
    spectrograms.append(spectrogram)
    labels.append(label)

Processando dados: 100%|██████████| 21968/21968 [07:22<00:00, 49.66data/s]


In [ ]:
spectrogram, label = encode_single_sample(dataset[0]['audio']['array'], dataset[0]['sentence'])

In [ ]:
spectrogram

<tf.Tensor: shape=(1219, 193), dtype=float32, numpy=
array([[ 1.6657089 ,  1.7006105 ,  1.8479182 , ..., -0.82513076,
        -0.83893055, -0.829094  ],
       [ 1.999373  ,  1.7483722 ,  1.4855099 , ..., -0.81034017,
        -0.80858064, -0.8089801 ],
       [ 0.7909164 ,  0.95239437,  1.2188578 , ..., -0.80961126,
        -0.8095434 , -0.80952334],
       ...,
       [ 0.2551299 ,  0.8557077 ,  0.9137746 , ..., -0.86999285,
        -0.87001526, -0.8697995 ],
       [ 0.6691177 ,  0.6066629 ,  0.76785856, ..., -0.87769675,
        -0.8774716 , -0.8778401 ],
       [ 0.65872556,  0.49569604,  0.69298553, ..., -0.87480515,
        -0.88802457, -0.87769204]], dtype=float32)>

In [ ]:
label

<tf.Tensor: shape=(44,), dtype=int64, numpy=
array([19,  9, 14, 20,  1, 43, 19,  5, 44,  6,  5, 12,  9, 26, 44,  3, 15,
       13, 44,  1, 44, 22,  9, 20, 34, 18,  9,  1, 44, 17, 21,  5, 44, 22,
       15,  3, 32, 44,  7,  1, 14,  8,  1, 42])>

In [84]:
# Dividir os dados em conjuntos de treinamento e validação
X_train, X_val, y_train, y_val = train_test_split(spectrograms, labels, test_size=0.1, random_state=42)

In [89]:
batch_size = 32

In [92]:
max_length = max(spec.shape[0] for spec in X_train)

In [93]:
max_length

3369

In [98]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(dataset["path"]), list(dataset["sentence"]))
)

In [91]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (X_train, y_train)
)

InvalidArgumentError: {{function_node __wrapped__Pack_N_19771_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [1900,193] != values[1].shape = [1806,193] [Op:Pack] name: component_0

In [90]:
teste = X_train.padded_batch(batch_size)

AttributeError: 'list' object has no attribute 'padded_batch'

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(dataset['audio']['array'], dataset['sentence'], test_size=0.10, random_state=42)

In [ ]:
# Split the dataset into training and validation sets. 90/10.
split = int(len(dataset) * 0.90)
# df_train = metadata_df[:split]
df_val = dataset[split:]

# print(f"Size of the training set: {len(df_train)}")
print(f"Size of the training set: {len(df_val)}")

Size of the training set: 12


In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(
    (list(dataset['audio']['array']), list(dataset['sentence']))
)